# 2-Import Data
In this notebook, we read data from the downloaded CSV files and Excel files, and import it into a SQLite database.

**Requirements:**

- Please run the `1-download-ved.ipynb` notebook first.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html)

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import os

from tqdm.notebook import tqdm
from sqlapi import VedDb

Set the data path and target file name.

In [ ]:
data_path = "./data"

The `read_data_frame` reads data from a single CSV file into a Pandas DataFrame.

In [ ]:
def read_data_frame(filename):
    columns = ['DayNum', 'VehId', 'Trip', 'Timestamp(ms)', 
               'Latitude[deg]', 'Longitude[deg]', 
               'Vehicle Speed[km/h]', 'MAF[g/sec]', 
               'Engine RPM[RPM]', 'Absolute Load[%]',
               'OAT[DegC]', 'Fuel Rate[L/hr]', 
               'Air Conditioning Power[kW]', 'Air Conditioning Power[Watts]',
               'Heater Power[Watts]', 'HV Battery Current[A]', 
               'HV Battery SOC[%]', 'HV Battery Voltage[V]',
               'Short Term Fuel Trim Bank 1[%]', 'Short Term Fuel Trim Bank 2[%]',
               'Long Term Fuel Trim Bank 1[%]', 'Long Term Fuel Trim Bank 2[%]'
              ]
    types = {'VehId': np.int64,
             'Trip': np.int64,
             'Timestamp(ms)': np.int64
            }
    df = pd.read_csv(filename, usecols=columns, dtype=types)
    return df

Enumerate the CSV files from the expanded data directory.

In [ ]:
files = [os.path.join(data_path, file) for file in tqdm(os.listdir(data_path)) if file.endswith(".csv")]

Create a `VedDb` object. This is the API to interface with the SQLite database.

In [ ]:
db = VedDb()

Iterate through the data files, import them into a Pandas DataFrame and then insert the signals into the database. Note that the signals are bulk-inserted, so the DataFrame is actually converted to a list of tuples.

In [ ]:
for file in tqdm(files):
    df = read_data_frame(file)
    
    signals = []
    for row in df.itertuples(index=False):
        signals.append(row)
        
    db.insert_signals(signals)

Now we can load the static data from the ICE & HEV vehicles.

In [ ]:
df_ice_hev = pd.read_excel("./ved/Data/VED_Static_Data_ICE&HEV.xlsx").replace('NO DATA', np.nan)

In [ ]:
df_ice_hev.head()

We follow by reading the PHEV and EV static data.

In [ ]:
df_phev_ev = pd.read_excel("./ved/Data/VED_Static_Data_PHEV&EV.xlsx").replace('NO DATA', np.nan)

In [ ]:
df_phev_ev.head()

Next, we collect all vehicle definitions in one list and bulk insert it into the database.

In [ ]:
vehicles = []

for row in df_ice_hev.itertuples(index=False):
    vehicles.append(row)
for row in df_phev_ev.itertuples(index=False):
    vehicles.append(row)
    
db.insert_vehicles(vehicles)

This creates the first image of the database. In subsequent notebooks we will use it to further analyse the data and derive some (hopefully) interesting models.

Generate data for the `move` table. Here we identify single trips per vehicle. Later we will use this table for joins when exploring the trajectory data.

In [ ]:
db.generate_moves()